In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, accuracy_score

In [3]:
base_path = Path('data/v2/')

X_train = np.load(base_path / 'train_images.npy').astype(np.float32)
Y_train = np.load(base_path / 'train_labels.npy')

X_test = np.load(base_path / 'test_public_images.npy').astype(np.float32)
Y_test = np.load(base_path / 'test_public_labels.npy')

classes, counts = np.unique(Y_train, return_counts=True)
counts_test = np.unique(Y_test, return_counts=True)[1]

In [4]:
classes_names = [
    "Avoine d'hiver",
    "Avoine de printemps",
    "Blé tendre d'hiver",
    "Blé tendre de printemps",
    "Colza d'hiver",
    "Luzerne déshydratée",
    "Luzerne",
    "Mélange de légumineuses et graminées",
    "Maïs",
    "Maïs ensilage",
    "Orge d'hiver",
    "Orge de printemps",
    "Prairie permanente - herbe",
    "Prairie permanente - ressources fourragères ligneuses",
    "Prairie en rotation longue",
    "Pomme de terre de consommation",
    "Prairie temporaire",
    "Soja",
    "Tournesol",
    "Triticale d'hiver"
]
months = ['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']

print(f'Each {X_train.shape[0]} training and {X_test.shape[0]} test samples have :\n'
      f'\t{X_train.shape[1]} images (one per month from Feb to Nov)\n'
      f'\t{X_train.shape[2]} channels (B08 Near Infrared, B04 Red, B03 Green)\n'
      f'\t{X_train.shape[3]} x {X_train.shape[4]} pixels')
print(f'X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}')
print(f'X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}')

classes_df = pd.DataFrame({
    'Class Name': classes_names,
    'N Samples in Train': counts,
    'N Samples in Test': counts_test,
    'Frequency in Train': counts / counts.sum(),
    'Frequency in Test': counts_test / counts_test.sum()
},index=classes)


Each 10000 training and 2500 test samples have :
	10 images (one per month from Feb to Nov)
	3 channels (B08 Near Infrared, B04 Red, B03 Green)
	32 x 32 pixels
X_train shape: (10000, 10, 3, 32, 32), Y_train shape: (10000,)
X_test shape: (2500, 10, 3, 32, 32), Y_test shape: (2500,)


In [5]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter

def create_simplified_dataset(X_data):

    simplified_data = []
    for sample_idx in range(X_data.shape[0]):
        for month_idx in range(X_data.shape[1]):
            image = X_data[sample_idx, month_idx, :, :, :]
            filtered_image = gaussian_filter(image, sigma=1)
            avg_rgb = np.mean(filtered_image, axis=(1, 2))
            simplified_data.append({
              'sample_id':sample_idx,
              'month':month_idx,
              'R': avg_rgb[0],
              'G': avg_rgb[1],
              'B': avg_rgb[2]
            })

    return pd.DataFrame(simplified_data)


simplified_train_df = create_simplified_dataset(X_train)
print(simplified_train_df.head())

simplified_test_df = create_simplified_dataset(X_test)
print(simplified_test_df.head())

   sample_id  month            R            G            B
0          0      0  2815.662109  2280.853271  1920.013916
1          0      1  3364.649414  2543.026611  2010.343506
2          0      2  3690.846680  2437.498291  1626.443115
3          0      3  4328.185059  2770.220215  1770.037842
4          0      4  3982.349365  3261.173828  2689.330078
   sample_id  month            R            G            B
0          0      0  2967.741211  2450.070312  2069.227295
1          0      1  4040.710205  3004.723145  2310.290527
2          0      2  4199.181641  2742.998047  1798.129150
3          0      3  4001.596191  2627.909180  1729.434082
4          0      4  3500.541504  2884.736816  2385.992920


In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(30,)),
  tf.keras.layers.Dense(128, activation='leaky_relu'),
  tf.keras.layers.Dense(20, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_4 (Flatten)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │         3,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,060 (90.08 KB)

 Trainable params: 23,060 (90.08 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from sklearn.metrics import classification_report, balanced_accuracy_score

# Préparation des données
X_train_nn = simplified_train_df.drop(columns=['sample_id', 'month']).values.reshape(-1, 10, 3)
X_test_nn = simplified_test_df.drop(columns=['sample_id', 'month']).values.reshape(-1, 10, 3)

# Aplatir les données pour le modèle dense
X_train_nn = X_train_nn.reshape(X_train_nn.shape[0], -1)  # Résultat : (10000, 30)
X_test_nn = X_test_nn.reshape(X_test_nn.shape[0], -1)    # Résultat : (N_test, 30)

print(f'X_train_nn shape: {X_train_nn.shape}, Y_train shape: {Y_train.shape}')

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraînement
model.fit(X_train_nn, Y_train, epochs=10)


X_train_nn shape: (10000, 30), Y_train shape: (10000,)
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 932us/step - accuracy: 0.3385 - loss: 337.5905
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.5567 - loss: 36.1836
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step - accuracy: 0.6086 - loss: 23.5664
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6274 - loss: 18.1901  
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - accuracy: 0.6345 - loss: 16.3684
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.6443 - loss: 13.4013
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.6608 - loss: 11.2292
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step - accuracy: 0.6708 - loss: 9.0264
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.6669 - loss: 8.5619
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.6857 - loss: 6.4765


In [21]:
# Évaluation du modèle
loss, accuracy = model.evaluate(X_test_nn, Y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Prédictions
Y_pred = np.argmax(model.predict(X_test_nn), axis=1)

# Calcul de la balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, Y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

# Rapport de classification
print(classification_report(Y_test, Y_pred))

Test Loss: 14.2492
Test Accuracy: 0.4716
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step
Balanced Accuracy: 0.2859
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00         4
           2       0.64      0.43      0.51       781
           3       0.56      0.85      0.67       108
           4       1.00      0.49      0.66       225
           5       0.53      0.74      0.62        39
           6       0.00      0.00      0.00        36
           7       0.14      0.36      0.20        84
           8       0.36      0.44      0.40       116
           9       0.00      0.00      0.00        11
          10       0.27      0.75      0.39       265
          11       0.61      0.17      0.27       235
          12       0.77      0.94      0.85       266
          13       1.00      0.10      0.18        41
          14       0.00      0.00      0.00        40
          15       0.16    

/home/edepoix/Documents/TPS/TPS 2A/Science des données et apprentissage/Projet classification/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/edepoix/Documents/TPS/TPS 2A/Science des données et apprentissage/Projet classification/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/edepoix/Documents/TPS/TPS 2A/Science des données et apprentissage/Projet classification/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarnin